# Tutorial 2: Interacting with Databases in Sessions

In this tutorial we will create and then interact with a *relational* SQLite database, using `sqlalchemy`'s `Session` object.

We will:

* Create a relational database containing two linked tables: "Objects" (with ID, ra and dec) and "Distances" (with ID and distance).
* Retrieve data into numpy arrays so that it can be used For Science.

This tutorial was adapted for astronomy from [the one at `pythoncentral.io`](http://pythoncentral.io/introductory-tutorial-python-sqlalchemy/).

### Requirements

You will need to have `sqlalchemy` installed.
```
pip install sqlalchemy
```

In [39]:
import sqlalchemy as sq
import os
import numpy as np

In the previous tutorial, we used a somewhat manual approach where we separately defined a Table in a database, a mapper that would connect a python object to the database, and the python object itself. This is tedious.

In this tutorial, we will use an alternative approach: declaratives. The advantage of using declaratives is that they allow us to define `Table`, mappers and python objects at once in a class definition. Let's see.

In [44]:
from sqlalchemy.ext.declarative import declarative_base

## Creating Database

Initialize a declarative.

In [45]:
Base = declarative_base()

Now, we create Python class objects for two linked tables: "Objects" and "Distances".

In [46]:
class Object(Base):
    __tablename__ = 'object'
    
    #Define columns like we did in the last notebook
    id = sq.Column(sq.Integer, primary_key=True)
    ra = sq.Column(sq.Float, nullable=False)
    dec = sq.Column(sq.Float, nullable=False)

    
class Distance(Base):
    __tablename__ = 'distance'
    
    #Again define columns like above. These columns
    #are normal python instance attributes.
    id = sq.Column(sq.Integer, primary_key=True)
    object_id = sq.Column(sq.Integer, sq.ForeignKey("object.id"), nullable=False)
    distance = sq.Column(sq.Float, nullable=False)

Create a local engine to store data, and remove one if it already exists.

In [47]:
dbfile = 'sessions.db'

try: os.remove(dbfile)
except: pass

In [48]:
engine = sq.create_engine('sqlite:///'+dbfile)

Now, create tables in the database that we defined above.

In [49]:
Base.metadata.create_all(engine)

Check the size of database. It is filled with metadata, but there aren't any entries yet.

In [51]:
!wc -c $dbfile

   12288 sessions.db


## Inserting Data

For all the subequent operations (insert, update, delete, etc.), we are going to use a `sessionmaker` that establishes all conversations with the database. A session represents a 'stagging zone' for all the objects loaded into the database, just like in git. Any changes won't be persisted into the database until they are committed.

Notice that we use ORM module of sqlalchemy to import sessionmaker. ORM stands for Object-relational mapping and is a technique to make relational data compatible with object oriented languages.

In [52]:
from sqlalchemy.orm import sessionmaker

Bind the engine to the metadata of the Base class so that the declaratives can be accessed through a DBSession instance.

In [53]:
Base.metadata.bind = engine

In [54]:
DBSession = sessionmaker(bind=engine)

In [55]:
session = DBSession()

Insert a new object into the database with `ra` and `dec` values of `10.23` and `-32.36` respectively. 

In [56]:
new_object = Object(ra=10.23, dec=-32.36)
session.add(new_object)
session.commit()

Create a 100 distance entries for the object we inserted above.

In [57]:
for i in range(100):
    new_distance = Distance(distance=4, object_id=1)
    session.add(new_distance)
    session.commit()

## Querying Data

Make a query for the first object in the database and see its attributes.

In [58]:
obj = session.query(Object).first()
obj.id, obj.ra, obj.dec

(1, 10.23, -32.36)

Retrieve all the distances of the first object by using `filter()` method.

In [59]:
dist = session.query(Distance).filter(Distance.object_id==1)
np.array([d.distance for d in dist])

array([ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.])